In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np
from datetime import datetime 

import csv
import requests

import time
import concurrent.futures

from pairs_selection.api_wrapper import alph_settings as alph, val_steps as val, alph_api_wrapper as wrap

#PCA, Clustering
from sklearn.cluster import KMeans, DBSCAN
from sklearn import preprocessing
from sklearn.decomposition import PCA

In [3]:
settings = alph(
    apikey = "MD27QTTVAK3AJBUQ", 
    site = "https://www.alphavantage.co/query?", 
    date_range = [datetime(2020, 8, 9), datetime(2022, 2, 20)]
    )

In [4]:
tker_dict = val.get_tkers(settings)

In [5]:
valid_lst = val.get_valid_tkers(tker_dict)

Excluded: FBM, Foundation Building Materials Inc
Excluded: RFUN, RIVERFRONT DYNAMIC UNCONSTRAINED INCOME ETF 
Excluded: CLBTW, Cellebrite DI Ltd - Warrants (30/08/2026)
Excluded: API, Advanced Photonix Inc
Excluded: CLMA, ICLIMA GLOBAL DECARBONIZATION TRANSITION LEADERS ETF 
Excluded: GSB, GlobalScape
Excluded: WNW, Meiwu Technology Company Ltd
Excluded: EQOSW, Eqonex Ltd - Warrants (26/03/2021)
Excluded: DWAC, Digital World Acquisition Corp - Class A
Excluded: PSFE-WS, Paysafe Ltd - Warrants (11/08/2025)
Excluded: HOLOW, MicroCloud Hologram Inc - Warrants (31/01/2028)
Excluded: AVIV, Avantis International Large Cap Value ETF
Excluded: LGV, Longview Acquisition Corp II - Class A
Excluded: DNKN, Dunkin Brands Group Inc
Excluded: TTTN, UP FINTECH CHINA-U.S. INTERNET TITANS ETF 
Excluded: MSPR, MSP Recovery Inc - Class A
Excluded: PUNK, SUBVERSIVE METAVERSE ETF 
Excluded: PTIXW, Protagenic Therapeutics Inc - Warrants (13/04/2025)
Excluded: AVRE, Avantis Real E state ETF
Excluded: BITE, Bi

In [6]:
test_df = pd.DataFrame(valid_lst)
display(test_df)
test_lst = test_df[0].tolist()
#print(test_lst)

,0,1,2,3,4,5,6
0,MS-P-I,Morgan Stanley,NYSE,Stock,2014-09-24,null,Active
1,OCN,Ocwen Financial Corp,NYSE,Stock,1996-09-25,null,Active
2,NVCR,NovoCure Ltd,NASDAQ,Stock,2015-10-02,null,Active
3,BWX,SPDR Bloomberg International Treasury Bond ETF,NYSE ARCA,ETF,2007-10-11,null,Active
4,HOMB,Home Bancshares Inc,NYSE,Stock,2006-06-23,null,Active
...,...,...,...,...,...,...,...
8051,PZC,Pimco California Municipal Income Fund III,NYSE,ETF,2002-10-29,null,Active
8052,AAMC,Altisource Asset Management Corp,NYSE MKT,Stock,2012-12-13,null,Active
8053,NXJ,Nuveen New Jersey Quality Municipal Income Fund,NYSE,ETF,2002-09-12,null,Active
8054,FLWS,1-800 Flowers.com Inc - Class A,NASDAQ,Stock,1999-08-03,null,Active


In [7]:
function = "TIME_SERIES_INTRADAY_EXTENDED"
ticker_lst = test_lst[:100]
interval = "60min"
slice = ["year1month1"] #[f"year1month{i}" for i in range(1, 4)] #first three months of intraday data
outputsize = "compact"
test_urls = wrap.intraday_url(settings, function, ticker_lst, interval, outputsize = outputsize, output = "lst")

s = time.time()

with concurrent.futures.ThreadPoolExecutor() as executor: 
    futures = []
    data = []
    for url in test_urls: 
        futures.append(executor.submit(wrap.get_csv_data, url = url))
    for future in concurrent.futures.as_completed(futures): 
        data.append(future.result())

e = time.time()
print(f'Total time elapsed: {e-s} seconds')

# s = time.time()
# result2 = []
# for i in test_urls: 
#     result2.append(wrap.get_csv_data(i))
# e = time.time()
# print(f'Total time elapsed: {e-s} seconds')

Total time elapsed: 10.338699102401733 seconds


In [8]:
print(test_urls[0])
print(len(data[4]))

https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=PY&interval=1min&outputsize=compact&adjusted=False&datatype=csv&apikey=MD27QTTVAK3AJBUQ
905


# PCA


In [8]:
#Maybe minute interval not v good since some stocks don't have such frequency
#Get the stock data with most number of recorded prices
# for i,v in enumerate(data):
#     tracker=0
#     ind=0
#     if len(v)>tracker:
#         tracker=len(v)
#     ind=i

#For each stock, get the length of data (no. of timestamps)
len_dic={}
for i,v in enumerate(data):
    len_dic[i]=len(v)

#Get index of 
ind=max(len_dic)
stock_nm=test_lst[ind]

#Get pct_change, and transpose series such that it is 
# test_price=pd.to_numeric(test_df['close']).pct_change()
# test_price=test_price[1:].to_frame().transpose()
# test_price

#This will set the timestamps (columns) for each stock, since it is the longest
#headr=pd.DataFrame(data[ind][1:],columns=data[ind][0])['time'].to_list()

In [12]:
data[2]


[['time', 'open', 'high', 'low', 'close', 'volume'],
 ['2022-10-26 16:00:00', '21.83', '21.854', '21.79', '21.854', '1434'],
 ['2022-10-26 14:00:00', '21.77', '21.77', '21.77', '21.77', '8650'],
 ['2022-10-26 12:00:00', '21.82', '21.82', '21.82', '21.82', '1400'],
 ['2022-10-25 16:00:00', '21.9637', '22.005', '21.9484', '22.005', '759'],
 ['2022-10-25 14:00:00', '21.98', '21.9885', '21.98', '21.9885', '380'],
 ['2022-10-24 16:00:00', '21.985', '21.985', '21.985', '21.985', '510'],
 ['2022-10-24 14:00:00', '21.92', '21.975', '21.92', '21.975', '720'],
 ['2022-10-21 16:00:00', '21.96', '21.96', '21.96', '21.96', '547'],
 ['2022-10-21 13:00:00', '21.9399', '21.94', '21.9399', '21.94', '230'],
 ['2022-10-20 16:00:00', '21.7668', '21.7668', '21.7591', '21.7591', '268'],
 ['2022-10-20 15:00:00', '21.7', '21.7', '21.7', '21.7', '188'],
 ['2022-10-20 14:00:00', '21.7538', '21.83', '21.7538', '21.83', '384'],
 ['2022-10-19 16:00:00', '21.81', '21.81', '21.81', '21.81', '217'],
 ['2022-10-19 14:

In [9]:
df_test=pd.DataFrame(data[ind][1:],columns=data[ind][0])
#df_test.head()
pd.pivot_table(df_test,values='close',columns='time',aggfunc=np.sum)


time,2022-09-27 08:00:00,2022-09-27 09:00:00,2022-09-27 10:00:00,2022-09-27 11:00:00,2022-09-27 12:00:00,2022-09-27 13:00:00,2022-09-27 14:00:00,2022-09-27 15:00:00,2022-09-27 16:00:00,2022-09-27 17:00:00,...,2022-10-26 11:00:00,2022-10-26 12:00:00,2022-10-26 13:00:00,2022-10-26 14:00:00,2022-10-26 15:00:00,2022-10-26 16:00:00,2022-10-26 17:00:00,2022-10-26 18:00:00,2022-10-26 19:00:00,2022-10-26 20:00:00
close,15.25,15.4,15.5203,15.52,15.25,15.16,15.145,15.2799,15.3,15.45,...,17.52,17.54,17.385,17.385,17.2896,17.27,17.27,17.33,17.26,17.3
